In [1]:
import numpy as np
import pandas as pd

In [2]:
# Lectura de los datos del ipsa
ruta_ipsa = '../ipsa_matriz_final.csv'
ipsa = pd.read_csv(ruta_ipsa)

In [3]:
# Vemos el head para confirmar que este todo bien 
ipsa.head()

,Unnamed: 0,fecha,cierre,apertura,max,min,var_un_dia,dia,mes,anho,var_dos_dias,var_tres_dias,var_semana,var_empresa_uno,var_empresa_dos,var_empresa_tres,dolar,cu,subio
0,0,30.09.2022,5035.38,5100.24,5109.40,5024.98,-1.27,30,9,22,-2.77,-3.30,-8.83,-0.18,-2.00,-0.80,0.80,1.35,True
1,1,29.09.2022,5100.24,5180.64,5191.90,5099.70,-1.55,29,9,22,-2.05,-3.82,-9.08,-4.33,-0.79,-0.39,-2.92,2.35,False
2,2,28.09.2022,5180.64,5178.75,5222.29,5140.17,0.04,28,9,22,-2.31,-3.51,-7.17,-2.19,-0.10,-0.28,-0.89,-0.30,False
3,3,27.09.2022,5178.75,5207.22,5249.16,5176.10,-0.55,27,9,22,-3.55,-3.04,-7.46,-1.17,-4.82,-0.93,2.46,-1.55,True
4,4,26.09.2022,5207.22,5303.06,5303.06,5207.09,-1.81,26,9,22,-2.50,-5.72,-7.21,-1.21,0.76,0.54,2.65,-3.59,False


In [4]:
# Vemos todas las columnas de la matriz
list(ipsa.columns)

['Unnamed: 0',
 'fecha',
 'cierre',
 'apertura',
 'max',
 'min',
 'var_un_dia',
 'dia',
 'mes',
 'anho',
 'var_dos_dias',
 'var_tres_dias',
 'var_semana',
 'var_empresa_uno',
 'var_empresa_dos',
 'var_empresa_tres',
 'dolar',
 'cu',
 'subio']

In [5]:
# Definimos cuales van a ser las variables en las que se va a basar el modelo
caracteristicas = ['dia','mes','anho','cierre', 'apertura', 'max', 'min', 'var_un_dia', 'var_dos_dias', 'var_tres_dias', 'var_semana', 'var_empresa_uno', 'var_empresa_dos', 'var_empresa_tres', 'dolar', 'cu']

In [6]:
# Pasamos las variables en las que se va a basar el modelo a numpy
X = ipsa.loc[:,caracteristicas].to_numpy()

In [7]:
# Definimos la etiqueta de la variable que se va a buscar predecir 
etiqueta = ['subio']

In [8]:
# Pasamos la varaible que se busca predecir a numpy
y = ipsa.loc[:,etiqueta].to_numpy()

In [9]:
# Que % de los dias subio el IPSA
sum(y)/float(len(y))*100

array([48.44444444])

In [10]:
# Se importan las librerias que se van a utilizar para hacerel modelo predictivo
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [11]:
# Se definen las matrices de prueba y las matrices de entrenamiento 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [12]:
# Se crea el modelo de decision tree
modelo = DecisionTreeClassifier(max_depth = 8)

In [13]:
# Se hace el fit del modelo con las matrices de entrenamiento
modelo.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=8)

In [14]:
# Se hace la prediccion de la matriz de prueba para posteriormente sacar las metricas del modelo
Y_pred = modelo.predict(X_test)

In [15]:
# Se importa la libreria necesaria para hacer las metricas del modelo
from sklearn.metrics import confusion_matrix

In [16]:
# Se definen los verdaderos negativos 'tn', los falsos positivos 'fp', los falsos negativos 'fn' y los verdaderos positivos 'tp' utilizando la 
# funcion confusion_matrix 
tn, fp, fn, tp = confusion_matrix(y_test, Y_pred).ravel()

In [17]:
# Se imprimen los valores definidos anteriormente 
print('tn: ' + str(tn))
print('fp: ' + str(fp))
print('fn: ' + str(fn))
print('tp: ' + str(tp))

tn: 66
fp: 87
fn: 43
tp: 86


In [18]:
# Se calcualan todos las metricas importantes tal y como fueron vistas en clases 
correctitud = (tp+tn)/float(tp+fp+fn+tn)
sensibilidad = (tp)/float(tp+fn)
especificidad = (tn)/float(fp+tn)
precision = (tp)/float(tp+fp)
tasa_real = (tp+fn)/float(tp+fp+fn+tn)
f1_score = 2*(precision*sensibilidad)/float(precision+sensibilidad)

In [19]:
# Se imprimen todas las metricas antes calculadas 
print("correctitud: " + str(np.round(correctitud*100,2)) + "%")
print("sensibilidad: " + str(np.round(sensibilidad*100,2)) + "%")
print("especificidad: " + str(np.round(especificidad*100,2)) + "%")
print("precision: " + str(np.round(precision*100,2)) + "%")
print("tasa_real: " + str(np.round(tasa_real*100,2)) + "%")
print("f1_score: " + str(np.round(f1_score*100,2)) + "%")

correctitud: 53.9%
sensibilidad: 66.67%
especificidad: 43.14%
precision: 49.71%
tasa_real: 45.74%
f1_score: 56.95%


In [20]:
# Se importa la libreria 
from sklearn.tree import export_graphviz

In [21]:
# Se crea el arbol de decision como un archivo .dot
with open("arbol_decision.dot", 'w') as archivo_dot:
    export_graphviz(modelo, out_file = archivo_dot, filled=True, rounded=True, special_characters=False, impurity=False, feature_names=caracteristicas)

In [22]:
# Se transforma el archivo .dot a un archivo .png
!dot -Tjpeg arbol_decision.dot -o arbol_decision.png